## Lesson 1B - IMDB Subwords 8K with Multi Layer LSTM

In [1]:
!pip install tensorflow==2.0.0-beta1

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

print(tf.__version__)

2.0.0-beta1


In [0]:
## Get the data

dataset, info = tfds.load('imdb_reviews/subwords8k',
                          with_info = True,
                          as_supervised = True)

train_dataset, test_dataset = dataset['train'], dataset['test']

In [0]:
tokenizer = info.features['text'].encoder

In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE,
                                           train_dataset.output_shapes)
test_dataset = test_dataset.padded_batch(BATCH_SIZE,
                                         test_dataset.output_shapes)

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,
                                                       return_sequences = True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 635,329
Trainable params: 635,329
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [10]:
NUM_EPOCHS = 10

history = model.fit(train_dataset,
                    epochs = NUM_EPOCHS,
                    validation_data = test_dataset)

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


      8/Unknown - 53s 7s/step - loss: 0.6937 - accuracy: 0.5059

KeyboardInterrupt: ignored

### Interrupted execution as each epoch would take more than half an hour to execute

In [0]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_' + string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_' + string])
  plt.show()

In [0]:
plot_graphs(history, 'accuracy')

In [0]:
plot_graphs(history, 'loss')